<font size=5, font color='blue'> stata_df is the dataframe downloaded through Stata API

In [1]:
import pandas as pd
pd.options.mode.chained_assignment = None

stata_df = pd.read_stata("TUR_1.dta")
stata_df.head()

,period,period_start_day,value,status,_frequency,dataset_code,dataset_name,frequency,indexed_at,provider_code,series_code,series_name,series_num
0,1994-01,1994-01-01,100.39,NaN,monthly,CPIREER,CPI Based Real Effective Exchange Rate (2003=100),M,2025-01-08T02:23:08.472Z,TCMB,cpiddree,CPI - Developed Countries Based Real Effective...,Master
1,1994-02,1994-02-01,86.02,NaN,monthly,CPIREER,CPI Based Real Effective Exchange Rate (2003=100),M,2025-01-08T02:23:08.472Z,TCMB,cpiddree,CPI - Developed Countries Based Real Effective...,Master
2,1994-03,1994-03-01,76.97,NaN,monthly,CPIREER,CPI Based Real Effective Exchange Rate (2003=100),M,2025-01-08T02:23:08.472Z,TCMB,cpiddree,CPI - Developed Countries Based Real Effective...,Master
3,1994-04,1994-04-01,61.31,NaN,monthly,CPIREER,CPI Based Real Effective Exchange Rate (2003=100),M,2025-01-08T02:23:08.472Z,TCMB,cpiddree,CPI - Developed Countries Based Real Effective...,Master
4,1994-05,1994-05-01,61.50,NaN,monthly,CPIREER,CPI Based Real Effective Exchange Rate (2003=100),M,2025-01-08T02:23:08.472Z,TCMB,cpiddree,CPI - Developed Countries Based Real Effective...,Master


<font size=5, font color='blue'> python_df is the dataframe downloaded through Python API. 

In [2]:
python_df = pd.read_pickle("TUR_1.pickle")
python_df.head()

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,frequency,Frequency
0,monthly,TCMB,CPI,Consumer Price Index (2003=100) (TURKSTAT),cpimtm,CPI (Month to Month % Changes),2005-01,2005-01-01,0.55,0.55,M,Monthly
1,monthly,TCMB,CPI,Consumer Price Index (2003=100) (TURKSTAT),cpimtm,CPI (Month to Month % Changes),2005-02,2005-02-01,0.02,0.02,M,Monthly
2,monthly,TCMB,CPI,Consumer Price Index (2003=100) (TURKSTAT),cpimtm,CPI (Month to Month % Changes),2005-03,2005-03-01,0.26,0.26,M,Monthly
3,monthly,TCMB,CPI,Consumer Price Index (2003=100) (TURKSTAT),cpimtm,CPI (Month to Month % Changes),2005-04,2005-04-01,0.71,0.71,M,Monthly
4,monthly,TCMB,CPI,Consumer Price Index (2003=100) (TURKSTAT),cpimtm,CPI (Month to Month % Changes),2005-05,2005-05-01,0.92,0.92,M,Monthly


<font size=5, font color='blue'> Check whether the two dataframes have the same number of rows, which means equal number of data points / observations. Column numbers of the two dataframes may differ due to the difference between Stata and Python APIs, and so do the order, datatypes and names of columns and the order of rows, which is why we don't require a complete match between python_df and stata_df.

In [3]:
stata_df.shape[0]==python_df.shape[0]

True

<font size=5, font color='blue'> ['period_start_day'] in stata_df is the most accurate timestamp assigned to each observation (row). In python_df its equivalent is ['period'], so we rename it to match the column name in stata_df. We match the datatype of the two columns by converting both of them to pandas datetime objects. We also convert the datatype of ['value'] in stata_df to numeric.

In [4]:
python_df.rename(columns={'period':'period_start_day'},inplace=True)
stata_df['period_start_day'] = pd.to_datetime(stata_df['period_start_day'])
python_df['period_start_day'] = pd.to_datetime(python_df['period_start_day'])
stata_df['value'] = pd.to_numeric(stata_df['value'],'coerce').astype('float64')
python_df['value'] = python_df['value'].astype('float64')

<font size=5, font color='blue'> Check whether ['period_start_day', 'series_code'] can identify a unique row in both stata_df and python_df, if not then there is a duplication or error in Dbnomics.

In [5]:
cols = ['period_start_day', 'series_code']
sum(stata_df.duplicated(subset=cols))+sum(python_df.duplicated(subset=cols)) == 0

True

<font size=5, font color='blue'> Then we sort the two dataframes by ['period_start_day', 'series_code']. This ensures observations in the two dataframes are in the same order. Finally we check whether the two sub-frames [['period_start_day','series_code','value']] sliced from stata_df and python_df are exactly the same.

In [6]:
stata_sub = stata_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
python_sub = python_df.sort_values(by=cols)[cols+['value']].reset_index(drop=True)
stata_sub.equals(python_sub)

True